# Implementing distilBERT with fastai and HuggingFace 🤗Transformers

This notebook is based off of [this great tutorial kernel](https://www.kaggle.com/maroberti/fastai-with-transformers-bert-roberta) and accompanying [article](https://medium.com/p/fastai-with-transformers-bert-roberta-xlnet-xlm-distilbert-4f41ee18ecb2?source=email-29c8f5cf1dc4--writer.postDistributed&sk=119c3e5d748b2827af3ea863faae6376) Check out the original kernel (and give an upvote!) for lots of other transformer models that can be easily implemented in these notebooks. 


If you've seen my [roBERTa kernel](https://www.kaggle.com/melissarajaram/roberta-fastai-huggingface-transformers) this one is going to be quite similar. Here I've chosen to use the distilBERT architecture because it's smaller, and I'll be able to train more of the paramerters of the model within the confines of the kernel. I've kept a lot of the original tutorial's text to explain what the code is doing.

**A major difference in this notebook and the RoBERTa (and previous versions) notebooks is the implementation of cross validation. Instead of training one model, I train 5 different models, and average the output. Due to time contraints, I will not be able to train each model for very long, but I'm hoping the average across all is better. **

## Google Quest Q&A Overview

This challenge is about questions and answers. 

In [question answering (QA)](https://en.wikipedia.org/wiki/Question_answering) systems are built that automatically answer questions posed by humans in a natural language. These computer systems excel at answering questions with single, verifiable answers. In contrast, humans are better at addressing subjective questions that require a deeper, multidimensional understanding of context.  

For the [Google QUEST Q&A Labeling competition](https://www.kaggle.com/c/google-quest-challenge/overview), we're tasked with predicting different subjective aspects of question-answering. The data for this competition includes questions and answers, and the task is to predict target values of 30 labels for each question-answer pair.Target labels with the prefix question_ relate to the question_title and/or question_body features in the data, and target labels with the prefix answer_ relate to the answer feature.

This is not a binary prediction challenge. Target labels are aggregated from multiple raters, and can have continuous values in the range [0,1]. Submissions are evaluated with the mean [Spearman's rank correlation coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient).


## Notebook Contents

Breifly, this notebook uses a transfer learning approach to aid in the question-answer classification task. The first **transfer learning** method applied to Natural Language Processing (NLP) was [Universal Language Model Fine-tuning for Text Classification](https://medium.com/r/?url=https%3A%2F%2Farxiv.org%2Fpdf%2F1801.06146.pdf).(ULMFiT) method. This method uses language models trained to predict the next word in a sequence, which is then used in a classification taks.[Viedo here.](https://course.fast.ai/videos/?lesson=4).

Recently, a new architecture called the **Transformer** (cf. [Attention is all you need](https://arxiv.org/abs/1706.03762)) has been shown to be powerful. Google (BERT, Transformer-XL, XLNet), Facebook (RoBERTa, XLM) or even OpenAI (GPT, GPT-2) all have pre-trained their own models with architectures based on the transformer. These pretrained models are availiable through the [HuggingFace](https://huggingface.co/) 🤗 [transformers library](https://github.com/huggingface/transformers). Formerly known as ``pytorch-transformers`` or ``pytorch-pretrained-bert``, this library has both pre-trained NLP models and additional utilities like tokenizers, optimizers and schedulers. 

This kernel uses the ``transformers`` library within the ``fastai`` framework. Specifically, I am using the [distilBERT model](https://towardsdatascience.com/bert-roberta-distilbert-xlnet-which-one-to-use-3d5ab82ba5f8). I've broken the process down into different parts:

1. Set Up and Data Loading
1. Specifying Data Preprocessing
1. Loading and Processing Data
1. Creating the Model
1. Training the Model
    - With cross validation (V3)
1. Predictions and Submission

# Set Up and Data Loading

This kernel uses fastai and Huggingface transformser. fastai is already installed on Kaggle, and [here](https://www.kaggle.com/c/tensorflow2-question-answering/discussion/117716) is a discussion post that shows how to get Huggingface installled. In total, we'll need three datasets: `sacremoses`, `transformers`, and `distilbert`. See the note below in the 'kernel technicality' cell on setting up the distilbert dataset.

In [ ]:
!pip install ../input/sacremoses/sacremoses-master 
!pip install ../input/transformers/transformers-master 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path 

import os

import torch
import torch.optim as optim

import random 

# fastai
from fastai import *
from fastai.text import *
from fastai.callbacks import *

# classification metric
from scipy.stats import spearmanr

# cross validation
from sklearn.model_selection import KFold

# transformers
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

This statement prints all of the directories in the /kaggle/input/ directory. This can be useful when trying to determine the path of the external datasets. If you are running into weird errors where you can't reach one of you dataset, even though you have attached it, check the output of this cell to double check the path that kaggle has assigned it.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

A utility function to set the seed for generating random numbers

In [ ]:
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [ ]:
seed=42
seed_all(seed)

**Kernel Technicality:**
To make this run completely within the Kaggle kernels, we need to upload the pretrained configuration, vocabulary and model as a dataset (i.e., MODEL_ROOT below). To create the dataset, I had to create a separate notebook, download the files from the internet, and use `.save_pretrained` to save them in the format HuggingFace was expecting.For example:

    MODEL_ROOT = Path("/wherever/distilbert-base-uncased")
    transformer_tokenizer = tokenizer_class.from_pretrained(pretrained_model_name)
    transformer_tokenizer.save_pretrained(MODEL_ROOT)

    config = config_class.from_pretrained(pretrained_model_name)
    config.num_labels = 30
    config.use_bfloat16 = use_fp16
    config.save_pretrained(MODEL_ROOT)

    transformer_model = model_class.from_pretrained(pretrained_model_name, config = config)
    transformer_model.save_pretrained(MODEL_ROOT)

In [ ]:
DATA_ROOT = Path("../input/google-quest-challenge/")
MODEL_ROOT = Path("../input/distilbertbaseuncased/")
#DATA_ROOT = Path.cwd() / 'data'
#MODEL_ROOT = Path.cwd() / 'distilbert'
train = pd.read_csv(DATA_ROOT / 'train.csv')
test = pd.read_csv(DATA_ROOT / 'test.csv')
sample_sub = pd.read_csv(DATA_ROOT / 'sample_submission.csv')
print(train.shape,test.shape)

The training data. In this kernel, I'll use the `question_title`, `question_body` and `answer` columns.

In [ ]:
train.head()

The predicted lables are in the columns of the sample submission. Note that some labels are with respect to the question, and some are with respect to the answer.

In [ ]:
labels = list(sample_sub.columns[1:].values)
for label in labels: print(label)

# Specifying Data Preprocessing 

When using pretrained models, the current data needs to be preprocessed in the same way as the data that trained the model. In ``transformers``, each model architecture is associated with 3 main types of classes:
* A **model class** to load/store a particular pre-train model.
* A **tokenizer class** to pre-process the data and make it compatible with a particular model.
* A **configuration class** to load/store the configuration of a particular model.

All these classes share a common class method ``from_pretrained(pretrained_model_name, ...)`` with all the other huggingface models. The parameter ``pretrained_model_name`` is a string with the shortcut name of a pre-trained model/tokenizer/configuration to load, e.g ``'bert-base-uncased'``. For a kaggle competition, we have to load/save the model in another step. We can find all the shortcut names in the transformers documentation [here](https://huggingface.co/transformers/pretrained_models.html#pretrained-models). Check out the [original tutorial](https://www.kaggle.com/maroberti/fastai-with-transformers-bert-roberta) for other MODEL_CLASSES names. 

In [ ]:
MODEL_CLASSES = {
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)
}
model_type = 'distilbert'
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]

This statement tells us the distilBERT models we could select. Here, I'm using the 'distilbert-base-uncased'. I've tried using the 'distilbert-base-uncased-distilled-squad', and had a lot of difficulty getting it to accurately classify. I'm not sure why.

In [ ]:
model_class.pretrained_model_archive_map.keys()

## Using the distilBERT tokenizer and numericalizer within fastai

Text data is preprocessed through tokenization and numericalization. To match the pretrained models, we need to use the same tokenization and numericalization as the model. Fortunately, the **tokenizer class** from ``transformers`` provides the correct pre-process tools that correspond to each pre-trained model. In ``fastai``, data pre-processing is performed during the creation of the ``DataBunch``. When creating a `DataBunch`, the tokenizer and numericalizer are passed in the 'processor' argument.

Therefore, the first step is to create a customized tokenize and numericalizer that use the distilBERT transformer tokenizer classes, and then create a custom processor which we can use to create the `DataBunch`. 

### Custom Tokenizer

A tokenizer takes the text and transforms it into tokens. The ``fastai`` documentation notes that: 
1. The [``TokenizeProcessor`` object](https://docs.fast.ai/text.data.html#TokenizeProcessor) takes as ``tokenizer`` argument a ``Tokenizer`` object.
2. The [``Tokenizer`` object](https://docs.fast.ai/text.transform.html#Tokenizer) takes as ``tok_func`` argument a ``BaseTokenizer`` object.
3. The [``BaseTokenizer`` object](https://docs.fast.ai/text.transform.html#BaseTokenizer) implement the function ``tokenizer(t:str) → List[str]`` that take a text ``t`` and returns the list of its tokens.

To use the distilBERT tokenizer, we create a new class ``TransformersBaseTokenizer`` that inherits from ``BaseTokenizer`` and overwrite a new ``tokenizer`` function. DistilBERT does not require a space to start the input string, so ``add_prefix_space`` is not used.

In [ ]:
class TransformersBaseTokenizer(BaseTokenizer):
    """Wrapper around PreTrainedTokenizer to be compatible with fast.ai"""
    def __init__(self, pretrained_tokenizer: PreTrainedTokenizer, model_type = 'bert', **kwargs):
        self._pretrained_tokenizer = pretrained_tokenizer
        self.max_seq_len = pretrained_tokenizer.max_len
        self.model_type = model_type

    def __call__(self, *args, **kwargs): 
        return self

    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length and add the spesial tokens"""
        CLS = self._pretrained_tokenizer.cls_token
        SEP = self._pretrained_tokenizer.sep_token
        if self.model_type in ['roberta']:
            tokens = self._pretrained_tokenizer.tokenize(t, add_prefix_space=True)[:self.max_seq_len - 2]
        else:
            tokens = self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2]
        return [CLS] + tokens + [SEP]

In [ ]:
transformer_tokenizer = tokenizer_class.from_pretrained(MODEL_ROOT)
transformer_base_tokenizer = TransformersBaseTokenizer(pretrained_tokenizer = transformer_tokenizer, model_type = model_type)
fastai_tokenizer = Tokenizer(tok_func = transformer_base_tokenizer, pre_rules=[], post_rules=[])

### Custom Numericalizer

The numericalizer takes the the tokens, and turns them into numbers. The ``fastai`` documentation notes that:
1. The [``NumericalizeProcessor``  object](https://docs.fast.ai/text.data.html#NumericalizeProcessor) takes as ``vocab`` argument a [``Vocab`` object](https://docs.fast.ai/text.transform.html#Vocab)

To use the distilBERT numericalizer, we create a new class ``TransformersVocab`` that inherits from ``Vocab`` and overwrite ``numericalize`` and ``textify`` functions.

In [ ]:
class TransformersVocab(Vocab):
    def __init__(self, tokenizer: PreTrainedTokenizer):
        super(TransformersVocab, self).__init__(itos = [])
        self.tokenizer = tokenizer
    
    def numericalize(self, t:Collection[str]) -> List[int]:
        "Convert a list of tokens `t` to their ids."
        return self.tokenizer.convert_tokens_to_ids(t)
        #return self.tokenizer.encode(t)

    def textify(self, nums:Collection[int], sep=' ') -> List[str]:
        "Convert a list of `nums` to their tokens."
        nums = np.array(nums).tolist()
        return sep.join(self.tokenizer.convert_ids_to_tokens(nums)) if sep is not None else self.tokenizer.convert_ids_to_tokens(nums)

### Custom processor

Now that we have our custom **tokenizer** and **numericalizer**, we can create the custom **processor**. Notice we are passing the ``include_bos = False`` and ``include_eos = False`` options. This is because ``fastai`` adds its own special tokens by default which interferes with the ``[CLS]`` and ``[SEP]`` tokens added by our custom tokenizer.

In [ ]:
transformer_vocab =  TransformersVocab(tokenizer = transformer_tokenizer)
numericalize_processor = NumericalizeProcessor(vocab=transformer_vocab)

tokenize_processor = TokenizeProcessor(tokenizer=fastai_tokenizer, include_bos=False, include_eos=False)

transformer_processor = [tokenize_processor, numericalize_processor]

# Loading and Processing Data

Now that we have a custom processor, which contains the custom tokenizer and numericalizer, we can create the `DataBunch`. During the DataBunch creation, we have to pay attention to set the processor argument to our new custom processor ``transformer_processor`` and manage correctly the padding. For RoBERTa, it's usually advised to pad the inputs on the right rather than the left.

In [ ]:
pad_first = bool(model_type in ['xlnet'])
pad_idx = transformer_tokenizer.pad_token_id

This kernel uses [the data block API](https://docs.fast.ai/data_block.html#The-data-block-API), to create the `DataBunch`. 

In the `DataBunch` creation, I have specified to use the 'question_title','question_body', and 'answer' columns as the training data.

In [ ]:
bs = 14

In [ ]:
databunch = (TextList.from_df(train, cols=['question_title','question_body','answer'], processor=transformer_processor)
             .split_by_rand_pct(0.1,seed=seed)
             .label_from_df(cols=labels)
             .add_test(test)
             .databunch(bs=bs, pad_first=pad_first, pad_idx=pad_idx))

Check batch and tokenizer. Because the distilBERT tokenizer was used, there are a some special tokens prefaced with `##`.

In [ ]:
print('[CLS] token :', transformer_tokenizer.cls_token)
print('[SEP] token :', transformer_tokenizer.sep_token)
print('[PAD] token :', transformer_tokenizer.pad_token)
databunch.show_batch()

Check batch and numericalizer :

In [ ]:
print('[CLS] id :', transformer_tokenizer.cls_token_id)
print('[SEP] id :', transformer_tokenizer.sep_token_id)
print('[PAD] id :', pad_idx)
test_one_batch = databunch.one_batch()[0]
print('Batch shape : ',test_one_batch.shape)
print(test_one_batch)

# Creating the Model

As mentioned [here](https://github.com/huggingface/transformers#models-always-output-tuples), the distilBERT model's forward method always outputs a ``tuple`` with various elements depending on the model and the configuration parameters. In our case, we are interested to access only to the logits. If we check out the [distilbert classification source](https://huggingface.co/transformers/_modules/transformers/modeling_distilbert.html#DistilBertForSequenceClassification), we find that the model is only outputting the logits, and so the information we want is in the [0] location. To access the logits, we create a custom model.

In [ ]:
# defining our model architecture 
class CustomTransformerModel(nn.Module):
    def __init__(self, transformer_model: PreTrainedModel):
        super(CustomTransformerModel,self).__init__()
        self.transformer = transformer_model
        
    def forward(self, input_ids, attention_mask=None):
        
        attention_mask = (input_ids!=0).type(input_ids.type()) # attention_mask for distilBERT
            
        logits = self.transformer(input_ids,
                                attention_mask = attention_mask)[0]   
        return logits

To make the transformer adapted to multiclass classification, we need to specify the number of labels before loading the pre-trained model.

In [ ]:
use_fp16 = False

config = config_class.from_pretrained(MODEL_ROOT)
config.num_labels = 30
config.use_bfloat16 = use_fp16

In [ ]:
transformer_model = model_class.from_pretrained(str(MODEL_ROOT), config = config)
custom_transformer_model = CustomTransformerModel(transformer_model = transformer_model)

### Custom evaluation metric

An important part of training is being able to see how the model is performing. In this competition,
>Submissions are evaluated on the mean column-wise Spearman's correlation coefficient. The Spearman's rank correlation is computed for each target column, and the mean of these values is calculated for the submission score.

Although scipy provides an implementation of Spearman's R, we also need to take the average across all of the target columns. Therefore, we need to create our own custom metric. [Others have noted](https://www.kaggle.com/c/google-quest-challenge/discussion/120072) that Scipy's implementation Spearman's Rho is kind of slow, and suggest a custom alternative. 

- `on_epoch_begin`: create empty numpy arrays to hold the predictions and targets
- `on_batch_end`: after each block, append the most recent output (predictions) and targets
- `on_epoch_end`: when the epoch is finished, compute Spearman's R on the columns, and then take the average

In [ ]:
class AvgSpearman(Callback):
    
    def on_epoch_begin(self, **kwargs):
        self.preds = np.empty( shape=(0, 30) )
        self.target = np.empty( shape=(0, 30) )
    
    def on_batch_end(self, last_output, last_target, **kwargs):
        self.preds = np.append(self.preds,last_output,axis=0)
        self.target = np.append(self.target,last_target,axis=0)
    
    def on_epoch_end(self, last_metrics, **kwargs):
        spearsum = 0
        for col in range(self.preds.shape[1]):
            spearsum += spearmanr(self.preds[:,col],self.target[:,col]).correlation
        res = spearsum / (self.preds.shape[1] + 1)
        return add_metrics(last_metrics, res)

### Fastai Learner with Custom Optimizer

In fastai, the `Learner` holds the data, model and other parameters, like the optimizer and evaluation metric(s). Since we're using transformers, we want to use an optimizer designed for them: the AdamW optimizer. This optimizer matches Pytorch Adam optimizer Api, therefore, it becomes straightforward to integrate it within ``fastai``. To reproduce BertAdam specific behavior, you have to set ``correct_bias = False``.

In [ ]:
from fastai.callbacks import *
from transformers import AdamW

learner = Learner(databunch, 
                  custom_transformer_model, 
                  opt_func = lambda input: AdamW(input,correct_bias=False), 
                  metrics=[AvgSpearman()])

# Show graph of learner stats and metrics after each epoch.
learner.callbacks.append(ShowGraph(learner))

# Put learn in FP16 precision mode. --> Not working in the tutorial
if use_fp16: learner = learner.to_fp16()
    
learner.save("untrained")

# Training Cross Validated Models

Now that we've created the Learner, we can train the model. During training, we are going to use techniques known to help in other classification tasks: **discriminative layer training**, **gradual unfreezing** and **slanted triangular learning rates**. The kernel tutorial author noted that he didn't find any documentation about influence of these techniques with transformers. I've used them because I hope these techniques are domain general. Anecdotally, I find a benefit to gradual unfreezing with respect to time. Training with more layers frozen takes less time because there are fewer parameters. 

To implement unfreezing, our model needs to be specified into different layer groups. ``fastai`` allows us to "split" the structure model into groups, [described here](https://docs.fast.ai/basic_train.html#Discriminative-layer-training).

To see the structure of the distilBERT model, look at the output of the following:

In [ ]:
print(learner.model);

Let's check how many layer groups we currently have:

In [ ]:
num_groups = len(learner.layer_groups)
print('Learner split in',num_groups,'groups')

One group won't allow us to unfreeze parts of the model. The tutorial kernel suggested to divide the distilBERT model in 8 blocks:
* 1 Embedding
* 6 transformer layers
* 1 classifier

In [ ]:
list_layers = [learner.model.transformer.distilbert.embeddings,
               learner.model.transformer.distilbert.transformer.layer[0],
               learner.model.transformer.distilbert.transformer.layer[1],
               learner.model.transformer.distilbert.transformer.layer[2],
               learner.model.transformer.distilbert.transformer.layer[3],
               learner.model.transformer.distilbert.transformer.layer[4],
               learner.model.transformer.distilbert.transformer.layer[5],
               learner.model.transformer.pre_classifier]

learner.split(list_layers);

Let's check that we now have 8 layer groups:

In [ ]:
num_groups = len(learner.layer_groups)
print('Learner split in',num_groups,'groups')

### Model Training

To train the model we will:
- Find an appropriate initial learning rate
- Use 5 fold cross validation
- Progressively unfreeze the layers while training

During all training, we use the **Slanted Triangular Learning Rates** with the `.fit_one_cycle` command, described [here](https://docs.fast.ai/callbacks.one_cycle.html). Because we have initially created the `DataBunch` with a small batch size, we can use the same `DataBunch` throughout the training process.

#### Find an appropriate initial learning rate

In [ ]:
seed_all(seed)
learner.freeze_to(-1)

In [ ]:
learner.lr_find()

In [ ]:
learner.recorder.plot(suggestion=True)

Due to randomness, there can be little differences in the learning rate. Based on a few runs on my computer, I've chosen 1e-4 for the starting learning rate. 

#### Cross Validated Training with progressive unfreezing

First, a utility function to making sure the output data us in the right order. As [noted in other tutorials](https://mlexplained.com/2019/05/13/a-tutorial-to-fine-tuning-bert-with-fast-ai/) the function ``get_preds`` does not return the test/validation elements in order by default. Therefore, we will have to resort the test elements into their correct order.

In [ ]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learner.get_preds(ds_type)[0].detach().cpu().numpy()
    sampler = [i for i in databunch.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    return preds[reverse_sampler, :]

Here, the layers to unfreeze, epochs and learning rates are all specified. I create a new column on the training dataframe to hold which items are in the validation set.

In [ ]:
unfreeze_layers = [-1,-3,-5]
learning_rates = [1e-4, 1e-5, 1e-6]
epochs = [5, 2, 1]
all_predictions = []
train['is_valid'] = False

# prepare to get a five fold split
kf = KFold(n_splits=5, random_state=42, shuffle=True)

In [ ]:
for ind, (tr, val) in enumerate(kf.split(train)):
    
    # designate the new training and validation set
    train['is_valid'][tr] = False
    train['is_valid'][val] = True
    
    # create the databunch
    databunch = (TextList.from_df(train, cols=['question_title','question_body','answer'], 
                                  processor=transformer_processor)
             .split_from_df(col='is_valid')
             .label_from_df(cols=labels)
             .add_test(test)
             .databunch(bs=bs, pad_first=pad_first, pad_idx=pad_idx))
    
    # create the learner
    learner = Learner(databunch, 
                  custom_transformer_model, 
                  opt_func = lambda input: AdamW(input,correct_bias=False), 
                  metrics=[AvgSpearman()])
    learner.callbacks.append(ShowGraph(learner))
    if use_fp16: learner = learner.to_fp16()
        
    # load empty weights and split the model into layers
    learner.load("untrained")
    learner.split(list_layers);
    
    # progressively unfreeze the layers and train
    for layer in range(0,len(unfreeze_layers)):
        learner.freeze_to(unfreeze_layers[layer])
        print('fold:',ind,'freezing to:',unfreeze_layers[layer],' - ',epochs[layer],'epochs')
        learner.fit_one_cycle(epochs[layer], 
                              max_lr=slice(learning_rates[layer]*0.95**num_groups, learning_rates[layer]),
                              moms=(0.85, 0.75))

    test_preds = get_preds_as_nparray(DatasetType.Test)
    all_predictions.append(test_preds)

# Predictions and Submission

In [ ]:
avg_preds = np.mean(all_predictions, axis=0)

In [ ]:
avg_preds.shape

In [ ]:
sample_submission = pd.read_csv(DATA_ROOT / 'sample_submission.csv')
sample_submission[labels] = avg_preds
sample_submission.to_csv("submission.csv", index=False)

We check the order

In [ ]:
test.head()

In [ ]:
sample_submission.head()

Thanks for looking through this kernel! I hope that it helps you understand transformers, and how to integrate Huggingface with fastai. 

Check out the original for some other cool architectures:
[Fastai with HuggingFace 🤗Transformers (BERT, RoBERTa, XLNet, XLM, DistilBERT)](https://www.kaggle.com/maroberti/fastai-with-transformers-bert-roberta)

## References
* [Fastai with HuggingFace 🤗Transformers (BERT, RoBERTa, XLNet, XLM, DistilBERT)](https://www.kaggle.com/maroberti/fastai-with-transformers-bert-roberta)
* Hugging Face, Transformers GitHub (Nov 2019), [https://github.com/huggingface/transformers](https://github.com/huggingface/transformers)
* Fast.ai, Fastai documentation (Nov 2019), [https://docs.fast.ai/text.html](https://docs.fast.ai/text.html)
* Jeremy Howard & Sebastian Ruder, Universal Language Model Fine-tuning for Text Classification (May 2018), [https://arxiv.org/abs/1801.06146](https://arxiv.org/abs/1801.06146)
* Keita Kurita's article : [A Tutorial to Fine-Tuning BERT with Fast AI](https://mlexplained.com/2019/05/13/a-tutorial-to-fine-tuning-bert-with-fast-ai/) (May 2019)
* Dev Sharma's article : [Using RoBERTa with Fastai for NLP](https://medium.com/analytics-vidhya/using-roberta-with-fastai-for-nlp-7ed3fed21f6c) (Sep 2019)